In [1]:
%matplotlib qt
import hyperspy.api as hs
from espm.estimators import SmoothNMF
import numpy as np

def expand_spectrum(spectrum) : 
    r"""
    Expand the navigation dimension of a spectrum to enable hyperspy decomposition.
    """
    a = np.tile(spectrum.data, (2, 1))
    temp_spectrum = hs.signals.Signal1D(a)
    temp_spectrum.set_signal_type('EDS_espm')
    d = spectrum.metadata.as_dictionary()
    temp_spectrum.metadata.add_dictionary(d)
    dd= spectrum.axes_manager[0].get_axis_dictionary()
    temp_spectrum.axes_manager.signal_axes[0].set(**dd)
    temp_spectrum.model_ = spectrum.model
    return temp_spectrum
    

WARNING | Hyperspy | Numba is not installed, falling back to non-accelerated implementation. (hyperspy.decorators:256)
WARNING | Hyperspy | Numba is not installed, falling back to non-accelerated implementation. (hyperspy.decorators:256)
WARNING | Hyperspy | Numba is not installed, falling back to non-accelerated implementation. (hyperspy.decorators:256)
WARNING | Hyperspy | Numba is not installed, falling back to non-accelerated implementation. (hyperspy.decorators:256)
WARNING | Hyperspy | Numba is not installed, falling back to non-accelerated implementation. (hyperspy.decorators:256)
WARNING | Hyperspy | Numba is not installed, falling back to non-accelerated implementation. (hyperspy.decorators:256)
WARNING | Hyperspy | Numba is not installed, falling back to non-accelerated implementation. (hyperspy.decorators:256)
WARNING | Hyperspy | Numba is not installed, falling back to non-accelerated implementation. (hyperspy.decorators:256)
WARNING | Hyperspy | Numba is not installed, fal

# I. Loading the data

### Notes

- Depending on the origin of the data you may want to modify this notebook : 
    - For example if the data were acquired using Velox, when using `hs.load()`, the returned object is probably a list. Print that list and select the EDS data.
- If you have 3D data (e.g. a spectrum image), you can still use this notebook. Do either of the following
    - Sum over all the pixels of the dataset using `spectrum = spim.sum()` where spim is your 3D dataset
    - Pick an area of the spectrum image and perform the analysis over the picked area. (see the last part of this notebook)

In [2]:
# We crop the signal dimension since our model can't deal with energy scales containing 0eV.
spectrum = hs.load().isig[0.1 : 20.0].sum()
# The data are changed from ints to floats. The algorithm can only work with floats.
spectrum.change_dtype('float64')
# To apply the methods of this package we change to object to the package type
spectrum.set_signal_type('EDS_espm')

# II. Setting metadata

It is very important to correctly fill the metadata. They are required for the model to work. Depending on your acquisition software, some fields are already filled, check the metadata first before completing what's missing. 

## Check your current metadata

In [3]:
spectrum.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = np.float64(0.0)
│       │       ├── elevation_angle = np.float64(35.0)
│       │       ├── energy_resolution_MnKa = np.float64(130.0)
│       │       ├── number_of_frames = np.int64(250)
│       │       ├── type = SDD_efficiency.txt
│       │       ├── width_intercept = 0.065
│       │       └── width_slope = 0.01
│       ├── Stage
│       │   ├── tilt_alpha = np.float64(0.284)
│       │   ├── tilt_beta = np.float64(-0.44)
│       │   ├── x = np.float64(-0.000134)
│       │   ├── y = np.float64(1e-05)
│       │   └── z = np.float64(-0.000163)
│       ├── beam_energy = np.float64(200.0)
│       ├── camera_length = np.float64(115.9)
│       ├── magnification = np.float64(7200000.0)
│       └── microscope = Titan
├── General
│   ├── FileIO
│   │   ├── 0
│   │   │   ├── hyperspy_version = 2.1.1
│   │   │   ├── io_plugin = rsciio.emd
│   │   │   ├── operation = load
│   │   │   └── timestamp = 2024-11-18T16:38:16.357404+01:00
│   │   ├── 1
│   │   │   ├── hyperspy_version = 2.1.1
│   │   │   ├── io_plugin = rsciio.hspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2024-11-19T14:23:59.740064+01:00
│   │   └── 2
│   │       ├── hyperspy_version = 2.2.0
│   │       ├── io_plugin = rsciio.hspy
│   │       ├── operation = load
│   │       └── timestamp = 2025-02-02T19:37:27.186339+01:00
│   ├── date = 2024-10-11
│   ├── original_filename = DSO.emd
│   ├── time = 11:00:13
│   ├── time_zone = CET
│   └── title = EDS
├── Sample
│   └── elements = ['O', 'Cu', 'Sc', 'Dy']
├── Signal
│   └── signal_type = EDS_espm
└── xray_db = 200keV_xrays.json

## Required metadata

### exspy metadata

Check exspy for more documentation on these functions

In [4]:
spectrum.set_microscope_parameters(elevation_angle=35.0,
                                   azimuth_angle= 0.0,
                                   tilt_stage= 0.0,
                                   beam_energy = 200,
                                   )
spectrum.metadata.Acquisition_instrument.TEM.Stage.tilt_beta = 0.0
spectrum.set_elements([ 'O', 'Sc', 'Dy'])


### espm metadata

- thickness : size of the sample along the beam direction in cm.
- density : density of the sample in g.cm^-3
- xray_db : X-ray cross-section database. Currently 3 energies are available : 100keV, 200keV and 300keV. Please reach for the devs or use emtables to generate different databases.
- width slope & width intercept : 
    - The energy width of caracteristic X-rays vary with energy. Typically the reference is the width of the Mn-Ka line. In espm (for now), the change in width is modeled using a straight line (slope + intercept). If necessary, you can get your own values by measuring the width of a bunch of caracteristic X-ray peaks.
- detector_type : It can be either a text file that contains energy vs detection efficiency from your constructor. There is one such curve available for espm : 'SDD_efficiency.txt'. Or you can build your own simplified detector model using espm. Please, contact the devs for help on that.

In [5]:
spectrum.set_analysis_parameters(thickness= 1e-5,
                                 density = 3.5,
                                 xray_db = '200keV_xrays.json',
                                 width_slope = 0.01,
                                 width_intercept = 0.065,
                                 detector_type = 'SDD_efficiency.txt'
                                 )

# Optional metadata

These fields are required to try the fit with the mass-thickness as an adjustable variable.

- geom_eff : geometric efficiency of the detector in sr (i.e. solid angle covered by the detector.). 

In [6]:
spectrum.set_analysis_parameters(geom_eff=1.0)

spectrum.set_microscope_parameters(real_time=100.0,
                                   beam_current = 1)

# Calibrate the spectrum

If necessary you can adjust the energy scale of your dataset. Two windows will pop : one with the x-ray lines labels and the other one on which you can select a range for calibration.

Please check exspy for additional documentation.

In [6]:
spectrum.plot(True)
spectrum.calibrate()

findfont: Font family ['STIXGeneral'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXGeneral'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXGeneral'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXGeneral'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXNonUnicode'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXNonUnicode'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXNonUnicode'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXSizeOneSym'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXSizeTwoSym'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXSizeThreeSym'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXSizeFourSym'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXSizeFiveSym'] not found. Falling back to DejaVu Sans.
findfont: Font family ['cmsy10'] not

# Select background windows

A plot of the spectrum will pop with several ROIs. You can click and drag the ROIs to select the range over which the bremsstrahlung will be calculated. Once you're done selecting the energy ranges, you can click on "Apply". You should see the bremsstrahlung model appear.

Later ou you input directly the energy ranges to skip the selection gui.

In [ ]:
spectrum.select_background_windows(num_windows=5)

# Build the EDXS model

The `build_G` function has an important keyword argument : "elements_dict". It is structured as  : {element : cutoff}, e.g. `{Cu  : 3.0}`. It means that the fitting of lines of an element below the cutoff energy will be separated from the lines above the cutoff.
The "elements_dict" can be useful in three ways : 
- Since Cu characteristic X-rays are often artefacts originating from the sample support, their absorption coefficient is very different from the studied sample. Thus separating K from L lines of Cu results in a better fit.
- For some elements like the transition metals, the absorption coefficient may depend massively on the material composition and/or structure. The low energy lines, that are more sensitive to absorption effects may thus also be hard to fit and separating them will probably improve the fit. 
- For heavy elements, the cross-section of M or above lines may be wrong. Separating them from the rest will probably improve the fit.

In [7]:
spectrum.build_G(ignored_elements = ['Cu'],elements_dict={ 'Dy' : 3.0})

In [26]:
fW = spectrum.set_fixed_W({'p0' : {'Dy' : 0.25, 'Sc' : 0.25}})

# Initialise the NMF estimator

The nmf algorithm below takes a few key arguments :
- n_components : it has to be 1 when analysing a spectrum
- max_iter : max number of iterations
- tol : convergence criterion, the decomposition stops when it is reached
- G : EDXS model used to compute NMF
- hspy_comp : compatibility with hyperspy, it has to be True when using hyperspy objects.

In [8]:
tspectrum = expand_spectrum(spectrum)
nmf = SmoothNMF(n_components=1, max_iter=1000, tol=1e-6, G = tspectrum.model, hspy_comp = True)

C:\Users\adteurtrie\AppData\Local\Temp\ipykernel_13476\1262936053.py:16: DeprecationWarning: use "HasTraits.trait_set" instead
  temp_spectrum.axes_manager.signal_axes[0].set(**dd)


# Run the decomposition

In [9]:
tspectrum.decomposition(algorithm=nmf)

c:\Users\adteurtrie\AppData\Local\anaconda3\envs\espm_test\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


IndexError: index 3 is out of bounds for axis 0 with size 3

In [4]:
tspectrum.print_concentration_report()

ValueError: Input has to be either atomic number, either chemical symbols

In [3]:
tspectrum.plot_1D_results(elements=['O', 'Sc', 'Dy'])

findfont: Font family ['STIXGeneral'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXGeneral'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXGeneral'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXGeneral'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXNonUnicode'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXNonUnicode'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXNonUnicode'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXSizeOneSym'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXSizeTwoSym'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXSizeThreeSym'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXSizeFourSym'] not found. Falling back to DejaVu Sans.
findfont: Font family ['STIXSizeFiveSym'] not found. Falling back to DejaVu Sans.
findfont: Font family ['cmsy10'] not